In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from functools import reduce
import json
from IPython.display import display, Markdown

In [2]:
sample_submission=pd.read_csv("../data/raw/sample_submission.csv")
specs=pd.read_csv("../data/raw/specs.csv")
test=pd.read_csv("../data/raw/test.csv")
train=pd.read_csv("../data/interim/train.csv")
train_labels=pd.read_csv("../data/raw/train_labels.csv")

Those games roughly correspond to assessments which should be predicted. Looks like activities are given not for learning, but for relaxation (like fireworks).

In [3]:
train.title[train.type=="Game"].unique()

array(['Scrub-A-Dub', 'All Star Sorting', 'Air Show', 'Crystals Rule',
       'Dino Drink', 'Bubble Bath', 'Dino Dive', 'Chow Time',
       'Pan Balance', 'Happy Camel', 'Leaf Leader'], dtype=object)

In [4]:
train.title[train.type=="Assessment"].unique()

array(['Mushroom Sorter (Assessment)', 'Bird Measurer (Assessment)',
       'Cauldron Filler (Assessment)', 'Cart Balancer (Assessment)',
       'Chest Sorter (Assessment)'], dtype=object)

In [5]:
pd.options.display.max_colwidth=999
games_description=pd.DataFrame(data=[
    {"game": "Chow Time", "target": "balance the scales", 
    "fields": ["scale_weight", "target_weight", "resources", "scale_contents"]},
    {"game": "All Star Sorting", "target": "sort dinosaurs by houses",
    "fields": ["dinosaurs", "houses", "dinosaurs_placed"]},
    {"game": "Dino Dive", "target": "select dinosaur(s) which raises water in pool to given level (memory + measure)", 
     "fields":["target_level", "target_water_level"]},
    {"game": "Scrub-A-Dub", "target": "find item with right size for animal several times", 
     "fields": ["round_target", "size"]},
    {"game": "Pan Balance", "target": "Balance weights",
     "fields": ["target_weight", "weights"]},
    {"game": "Dino Drink", "target": "match sizes of eggs with holes",
    "fields": ["holes"]},
    {"game": "Air Show", "target": "Move dinousaur N spaces (into target_distances)", 
    "fields": ["distance", "target_distances"]},
    {"game": "Bubble Bath", "target": "Find container with right size", 
    "fields": ["target_containers", "containers"]},
    {"game": "Crystals Rule", "target": "Measure things",
    "fields": ["group", "target_size"]},
    {"game": "Happy Camel", "target": "Find bowl with toy using scales", 
     "fields": ["bowl_id", "scale_contents"]},
    {"game": "Leaf Leader", "target": "Balance scales",
     "fields": ["target_weight", "weight"]},
    
])
games_description

,game,target,fields
0,Chow Time,balance the scales,"[scale_weight, target_weight, resources, scale_contents]"
1,All Star Sorting,sort dinosaurs by houses,"[dinosaurs, houses, dinosaurs_placed]"
2,Dino Dive,select dinosaur(s) which raises water in pool to given level (memory + measure),"[target_level, target_water_level]"
3,Scrub-A-Dub,find item with right size for animal several times,"[round_target, size]"
4,Pan Balance,Balance weights,"[target_weight, weights]"
5,Dino Drink,match sizes of eggs with holes,[holes]
6,Air Show,Move dinousaur N spaces (into target_distances),"[distance, target_distances]"
7,Bubble Bath,Find container with right size,"[target_containers, containers]"
8,Crystals Rule,Measure things,"[group, target_size]"
9,Happy Camel,Find bowl with toy using scales,"[bowl_id, scale_contents]"


In [6]:
assessment_description=pd.DataFrame(data=[
    {"game": "Mushroom Sorter (Assessment)", "target": "Sort 3 mushrooms",
     "fields": ["stumps"]},
    {"game": "Bird Measurer (Assessment)", "target": "Measure 3 birds height",
     "fields": ["bird_height", "caterpillar", "caterpillars"]},
    {"game": "Cauldron Filler (Assessment)", "target": "Match buckets with cauldrons, then choose right bucket",
     "fields": ["bucket", "cauldron", "target_bucket"]},
    {"game": "Cart Balancer (Assessment)", "target": "Balance scales", 
     "fields": ["crystals", "left", "right"]},
    {"game": "Chest Sorter (Assessment)", "target": "Order chests using pan balance",
     "fields": ["left", "right", "pillars"]}
])
assessment_description

,game,target,fields
0,Mushroom Sorter (Assessment),Sort 3 mushrooms,[stumps]
1,Bird Measurer (Assessment),Measure 3 birds height,"[bird_height, caterpillar, caterpillars]"
2,Cauldron Filler (Assessment),"Match buckets with cauldrons, then choose right bucket","[bucket, cauldron, target_bucket]"
3,Cart Balancer (Assessment),Balance scales,"[crystals, left, right]"
4,Chest Sorter (Assessment),Order chests using pan balance,"[left, right, pillars]"


In [8]:
def show_game_session(df, title, filter_cols=tuple()):
    random_row = df[df.title == title].sample(1)
    installation_id = random_row.installation_id.iloc[0]
    session_id = random_row.game_session.iloc[0]
    show_player_sessions(df, installation_id, session_id, filter_cols)


def show_player_sessions(df, installation_id, session_id=None, filter_cols=tuple()):
    df = df[df.installation_id == installation_id].copy()
    game_sessions = [session_id] if session_id else df.game_session.unique()
    for game_session in game_sessions:
        session = df[df.game_session==game_session].copy()
        display(Markdown(describe_session(game_session, session)))
        with pd.option_context('display.max_rows', 999, 'display.max_columns', 999, 
                               'display.max_colwidth', 999):
            display(format_session(session, filter_cols))


def describe_session(game_session, session):
    head = session.sort_values("event_count").head(1)
    if "version" in json.loads(head.event_data.iloc[0]):
        version = json.loads(head.event_data.iloc[0])["version"]
    else:
        version = "-"
    return f"__{game_session} {head.title.iloc[0]} {head.type.iloc[0]} {head.world.iloc[0]} v{version}__ "


def format_session(session, filter_cols):
    session = session.sort_values("event_count")
    session["timestamp"]=session["timestamp"].str[:19]
    session["event_data"] = session.event_data.apply(json.loads)
    columns = sorted(reduce(set.__or__, session.event_data.apply(lambda d: set(d.keys())).values))
    for c in columns:
        session[c] = session.event_data.apply(lambda x: x.get(c, '-'))
    session["version"]=None
    session = session.drop(["game_session", "installation_id", "event_data", "game_time", 
                            "title", "type", "world", "version"] + [c for c in filter_cols if c in session.columns], axis=1)
    session.set_index("event_count", inplace=True)
    
    return session


for game in set(train.title[train.type=="Game"].unique()):
    show_game_session(train, game, ["coordinates"])

__50d53718ed14d671 Leaf Leader Game CRYSTALCAVES v1.0__ 

,Unnamed: 1,event_id,timestamp,event_code,index,correct,description,dinosaur,dinosaur_count,dinosaur_weight,duration,identifier,media_type,misses,mode,round,target_weight,total_duration,tutorial_step,weight
event_count,,,,,,,,,,,,,,,,,,,,
1,1159,8ac7cce4,2019-09-30T17:46:27,2000,5125764.0,-,-,-,-,-,-,-,-,-,-,0,-,-,-,-
2,1160,fd20ea40,2019-09-30T17:46:29,4010,5125765.0,-,-,-,-,-,-,-,-,-,-,0,-,-,-,-
3,1161,fd20ea40,2019-09-30T17:46:30,4010,5125766.0,-,-,-,-,-,-,-,-,-,-,0,-,-,-,-
4,1162,29f54413,2019-09-30T17:46:31,2060,5125767.0,-,-,-,-,-,-,-,-,-,-,0,-,-,-,-
5,1163,33505eae,2019-09-30T17:46:31,3010,5125768.0,-,"We've got to be in balance with the other team. This one's all about weight. Weight is a measure of how heavy something is. In this game small dinosaurs weigh a little, and big dinosaurs weigh a lot. This means BIG dinosaurs weigh MORE than small dinosaurs Tap the dinosaurs to add them to the rope. Tap them again to take them off the rope. The most dinosaurs you can add to the rope is 4. Add dinosaurs to the rope until the weight of your team equals the weight of the other team! Remember the most you can use is 4! You'll know the two sides are equal when the leaf is right above the dinosaur's head. So close he can eat it!",-,-,-,-,"WeveGotToBeInBalance,SmallWeighBigWeigh,ClickToAddRemoveRope_touch,MostIs4,AddDinosToRopeUntilEqual,YouKnowTheyAreEqualWhen",animation,-,-,0,-,44733,-,-
6,1164,7dfe6d8a,2019-09-30T17:46:39,4070,5125769.0,-,-,-,-,-,-,-,-,-,-,0,-,-,-,-
7,1166,2a512369,2019-09-30T17:46:41,3110,5125771.0,-,"We've got to be in balance with the other team. This one's all about weight. Weight is a measure of how heavy something is. In this game small dinosaurs weigh a little, and big dinosaurs weigh a lot. This means BIG dinosaurs weigh MORE than small dinosaurs Tap the dinosaurs to add them to the rope. Tap them again to take them off the rope. The most dinosaurs you can add to the rope is 4. Add dinosaurs to the rope until the weight of your team equals the weight of the other team! Remember the most you can use is 4! You'll know the two sides are equal when the leaf is right above the dinosaur's head. So close he can eat it!",-,-,-,9669,"WeveGotToBeInBalance,SmallWeighBigWeigh,ClickToAddRemoveRope_touch,MostIs4,AddDinosToRopeUntilEqual,YouKnowTheyAreEqualWhen",animation,-,-,0,-,-,-,-
8,1165,53c6e11a,2019-09-30T17:46:41,2075,5125770.0,-,-,-,-,-,9669,-,-,-,-,0,-,-,2,-
9,1167,f32856e4,2019-09-30T17:46:41,2020,5125772.0,-,-,-,-,-,-,-,-,-,-,1,6,-,-,-


__578dc5dac32b8065 Happy Camel Game CRYSTALCAVES v1.0__ 

,Unnamed: 1,event_id,timestamp,event_code,index,bowl_id,bowls,correct,description,duration,has_toy,identifier,media_type,misses,movie_id,position,round,scale_contents,source,time_played,total_bowls,total_duration
event_count,,,,,,,,,,,,,,,,,,,,,,
1,14903,d9c005dd,2019-09-21T03:23:07,2000,3363256.0,-,-,-,-,-,-,-,-,-,-,-,0,-,-,-,-,-
2,14904,abc5811c,2019-09-21T03:23:11,4010,3363257.0,-,-,-,-,-,-,-,-,-,-,-,0,-,-,-,-,-
3,14905,d51b1749,2019-09-21T03:23:13,2080,3363258.0,-,-,-,"It's the super powerful, Queen Cleopatra! What's the problem, Cle? My silly camel Epidermis has lost his toys in his food. Can you help us find them?",14933333,-,-,-,-,Intro,-,0,-,-,-,-,-
4,14906,c189aaf2,2019-09-21T03:23:28,2083,3363259.0,-,-,-,"It's the super powerful, Queen Cleopatra! What's the problem, Cle? My silly camel Epidermis has lost his toys in his food. Can you help us find them?",14933333,-,-,-,-,Intro,-,0,-,-,-,-,-
5,14907,c2baf0bd,2019-09-21T03:23:31,2020,3363260.0,-,"[{'id': 1, 'position': [209, 422, 1014, 541], 'toy': False}, {'id': 2, 'position': [336, 422, 1014, 541], 'toy': True}]",-,-,-,-,-,-,-,-,-,1,-,-,-,2,-
6,14908,8d7e386c,2019-09-21T03:23:31,3010,3363261.0,-,-,-,"Epidermis' toy is in one of these bowls of camel food! But which one? We shall find the toys in no time thanks to my pan balance! This amazing device for measuring weight. First, move a bowl to one of the pans.",-,-,"Cleo_EpidermisToy,Cleo_PanBalance,Cleo_MoveBowlToPan",audio,-,-,-,1,-,-,-,-,20920
7,14909,3d8c61b0,2019-09-21T03:23:33,4030,3363262.0,2,-,-,-,-,True,-,-,-,-,2,1,"[empty, empty]",resources,-,-,-
8,14910,6bf9e3e1,2019-09-21T03:23:34,4040,3363263.0,2,-,-,-,-,True,-,-,-,-,2,1,"[empty, toy]",resources,-,-,-
9,14911,69fdac0a,2019-09-21T03:23:34,3110,3363264.0,-,-,-,"Epidermis' toy is in one of these bowls of camel food! But which one? We shall find the toys in no time thanks to my pan balance! This amazing device for measuring weight. First, move a bowl to one of the pans.",3684,-,"Cleo_EpidermisToy,Cleo_PanBalance,Cleo_MoveBowlToPan",audio,-,-,-,1,-,-,-,-,-


__4e44da0ff3f8d6fd Air Show Game TREETOPCITY v1__ 

...

,Unnamed: 1,event_id,timestamp,event_code,index,correct,description,diet,dinosaur,duration,identifier,media_type,misses,resources,round,scale_contents,scale_weight,source,target_weight,total_duration,weight
event_count,,,,,,,,,,,,,,,,,,,,,
1,334,7d093bf9,2019-08-12T21:08:21,2000,6640916.0,-,-,-,-,-,-,-,-,-,0,-,-,-,-,-,-
2,335,f93fc684,2019-08-12T21:08:25,4010,6640917.0,-,-,-,-,-,-,-,-,-,0,-,-,-,-,-,-
3,336,7ec0c298,2019-08-12T21:08:27,3010,6640918.0,-,"It's Chow Time! We have some VERY hungry dinosaurs waiting. Let's feed them! Big dinosaurs eat a lot of food, small dinosaurs only eat a little!",-,-,-,"1431,1401b",animation,-,-,0,-,-,-,-,10333,-
4,337,0d1da71f,2019-08-12T21:08:38,3110,6640919.0,-,"It's Chow Time! We have some VERY hungry dinosaurs waiting. Let's feed them! Big dinosaurs eat a lot of food, small dinosaurs only eat a little!",-,-,10417,"1431,1401b",animation,-,-,0,-,-,-,-,-,-
5,338,63f13dd7,2019-08-12T21:08:38,2020,6640920.0,-,-,carnivore,buddy,-,-,-,-,"[3, 1, 1, 3]",1,-,-,-,4,-,-
6,339,7ec0c298,2019-08-12T21:08:38,3010,6640921.0,-,"To make the sure dinosaur eats the right amount tap and drag the food to the scale, until it's balanced! The scale is balanced when the two arrows are lined up, and green You can add, or take away, food if the scale is not balanced.",-,-,-,"1434,1435REVA,3200",animation,-,-,1,-,-,-,-,16000,-
7,340,0d1da71f,2019-08-12T21:08:54,3110,6640922.0,-,"To make the sure dinosaur eats the right amount tap and drag the food to the scale, until it's balanced! The scale is balanced when the two arrows are lined up, and green You can add, or take away, food if the scale is not balanced.",-,-,16145,"1434,1435REVA,3200",animation,-,-,1,-,-,-,-,-,-
8,341,0330ab6a,2019-08-12T21:08:55,3020,6640923.0,-,"That's not enough food. Try putting a piece on the scale, to make that side heavier!",-,-,-,1438,animation,-,-,1,-,-,-,-,5875,-
9,342,cfbd47c8,2019-08-12T21:08:56,4030,6640924.0,-,-,-,-,-,-,-,-,"[3, 3]",1,-,-,resources,-,-,1


__fbf1f62ef3aaa89d Crystals Rule Game TREETOPCITY v1.0__ 

,Unnamed: 1,event_id,timestamp,event_code,index,correct,description,duration,group,identifier,media_type,misses,object_type,position,resources,round,target_size,total_duration
event_count,,,,,,,,,,,,,,,,,,
1,355,48349b14,2019-08-15T03:59:06,2000,5106156.0,-,-,-,-,-,-,-,-,-,-,0,-,-
2,356,cc5087a3,2019-08-15T03:59:08,4010,5106157.0,-,-,-,-,-,-,-,-,-,-,0,-,-
3,357,5154fc30,2019-08-15T03:59:10,3010,5106158.0,-,"Let's measure some big rocks with something smaller. You can move the object under the rock to figure out how long the rock is Once you know how many objects you need, tap on the right number.",-,-,instructions_touch,animation,-,-,-,-,0,-,12400
4,358,5e3ea25a,2019-08-15T03:59:14,4070,5106159.0,-,-,-,-,-,-,-,-,-,-,0,-,-
5,359,5e3ea25a,2019-08-15T03:59:15,4070,5106160.0,-,-,-,-,-,-,-,-,-,-,0,-,-
6,360,5e3ea25a,2019-08-15T03:59:16,4070,5106161.0,-,-,-,-,-,-,-,-,-,-,0,-,-
7,361,5e3ea25a,2019-08-15T03:59:18,4070,5106162.0,-,-,-,-,-,-,-,-,-,-,0,-,-
8,362,5e3ea25a,2019-08-15T03:59:18,4070,5106163.0,-,-,-,-,-,-,-,-,-,-,0,-,-
9,363,5e3ea25a,2019-08-15T03:59:19,4070,5106164.0,-,-,-,-,-,-,-,-,-,-,0,-,-


__1a5f90a72f02c233 Pan Balance Game CRYSTALCAVES v1.0__ 

,Unnamed: 1,event_id,timestamp,event_code,index,correct,description,duration,identifier,location,media_type,misses,round,scale_weights,starting_weights,table_weights,target_weight,total_duration,weights
event_count,,,,,,,,,,,,,,,,,,,
1,1297,9c5ef70c,2019-08-01T16:45:18,2000,6906140.0,-,-,-,-,-,-,-,0,-,-,-,-,-,-
2,1298,0086365d,2019-08-01T16:45:20,4010,6906141.0,-,-,-,-,-,-,-,0,-,-,-,-,-,-
3,1299,a592d54e,2019-08-01T16:45:21,2020,6906142.0,-,-,-,-,-,-,-,1,-,2,4,1,-,-
4,1300,6cf7d25c,2019-08-01T16:45:21,3010,6906143.0,-,"This is a pan balance. You use it to tell how heavy something is. Put rocks on one side, and copper weights on the other side, and once the two sides are the same, press the green check button!",-,INSTRCOMPLETE,-,animation,-,1,-,-,-,-,12866,-
5,1301,f3cd5473,2019-08-01T16:45:23,4070,6906144.0,-,-,-,-,-,-,-,1,-,-,-,-,-,-
6,1302,f3cd5473,2019-08-01T16:45:29,4070,6906145.0,-,-,-,-,-,-,-,1,-,-,-,-,-,-
7,1303,f3cd5473,2019-08-01T16:45:30,4070,6906146.0,-,-,-,-,-,-,-,1,-,-,-,-,-,-
8,1304,f3cd5473,2019-08-01T16:45:31,4070,6906147.0,-,-,-,-,-,-,-,1,-,-,-,-,-,-
9,1305,f3cd5473,2019-08-01T16:45:31,4070,6906148.0,-,-,-,-,-,-,-,1,-,-,-,-,-,-


In [10]:
for game in set(test.title[test.type=="Assessment"].unique())-set(assessment_description.game.values):
    show_game_session(train, game, ["coordinates"])